In [ ]:
import pandas as pd
import os
os.environ["R_HOME"] = "/Library/Frameworks/R.framework/Resources"
from pymer4.models import Lmer
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.nonparametric import rank_compare_2indep
from lifelines import CoxPHFitter

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/Mastertabelle_filtered.xlsx'

In [ ]:
df = pd.read_excel(data_path)

In [ ]:
df = df[df.Comment != 'Reanimation']

In [ ]:
df.head()

In [ ]:
outcomes_of_interest = [
    "ICU [d]",
    "Ventilator [d]",
    "Hospital stay [d]",
    "28 d Survival [y/n]",
    "ITN ass. Pneumonia [y/n]"
    ]

In [ ]:
time_columns =["ICU [d]",
    "Ventilator [d]",
    "Hospital stay [d]"]
# in every column, if value ends with +, remove it
for col in time_columns:
    df[col] = pd.to_numeric(df[col].apply(lambda x: str(x).replace("+", '')), errors='coerce')


In [ ]:
df['Sverweis_Geschlecht_encoded'] = df['Sverweis_Geschlecht'].map({'männlich': 0, 'weiblich': 1})
df['28 d Survival [y/n]_encoded'] = df['28 d Survival [y/n]'].map({'n': 0, 'y': 1})
df['ICU Survival [y/n]_encoded'] = df['ICU Survival [y/n]'].map({'n': 0, 'y': 1})
df['Spital Survival [y/n]_encoded'] = df['Spital Survival [y/n]'].map({'n': 0, 'y': 1})
df['ITN ass. Pneumonia [y/n]_encoded'] = df['ITN ass. Pneumonia [y/n]'].map({'n': 0, 'y': 1})

In [ ]:
df.columns = df.columns.str.replace(' ', '_')

In [ ]:
# df['GCS_cat'] = pd.Categorical(df['GCS_(Erstbefund)'], categories=[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], ordered=True)
# # then convert to strings categories
# df['GCS_cat'] = df['GCS_cat'].astype(str)
# # and then convert to strings categories
# df['GCS_cat'] = pd.Categorical(df['GCS_cat'], categories=['3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'], ordered=True)

In [ ]:
df['Einteilung_encoded'] = df['Einteilung_(Hauptdiagnose)'].map({'Krankheit': 0, 'Unfall': 1})

## Influence of GCS on long-term outcoems

#### ICU LOS

In [ ]:
# # icu los
# icu_los_df = df[["ICU_[d]", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
# icu_los_df.columns = ['ICU_los', 'GCS', 'sex', 'age', 'category']
# print(f'number of nan (excluded): {icu_los_df.isna().sum().sum()}')
# icu_los_df.dropna(inplace=True)
#
#
# overall_icu_los_model = Lmer(f"ICU_los ~ GCS + sex + age + (1|category)", data=icu_los_df, family='gaussian')
# overall_icu_los_res = overall_icu_los_model.fit()
# print(overall_icu_los_res)

In [ ]:
# survival model
cph = CoxPHFitter()
icu_los_df = df[["ICU_[d]", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded', 'ICU_Survival_[y/n]_encoded']]
icu_los_df.columns = ['ICU_los', 'GCS', 'sex', 'age', 'category', 'icu_survival']
icu_los_df['icu_death'] = -1 * icu_los_df['icu_survival'] + 1
icu_los_df.drop(columns='icu_survival', inplace=True)

print(f'number of nan (excluded): {icu_los_df.isna().sum().sum()}')
icu_los_df.dropna(inplace=True)

cph.fit(icu_los_df, duration_col='ICU_los', event_col='icu_death')
cph.print_summary()

In [ ]:
# unadjusted survival model
cph = CoxPHFitter()
icu_los_df = df[["ICU_[d]", 'GCS_(Erstbefund)', 'ICU_Survival_[y/n]_encoded']]
icu_los_df.columns = ['ICU_los', 'GCS', 'icu_survival']
icu_los_df['icu_death'] = -1 * icu_los_df['icu_survival'] + 1
icu_los_df.drop(columns='icu_survival', inplace=True)

print(f'number of nan (excluded): {icu_los_df.isna().sum().sum()}')
icu_los_df.dropna(inplace=True)

cph.fit(icu_los_df, duration_col='ICU_los', event_col='icu_death')
cph.print_summary()

In [ ]:
# # unadjusted model
# unadjusted_icu_los_model = sm.OLS(icu_los_df['ICU_los'], sm.add_constant(icu_los_df[['GCS']]))
# unadjusted_icu_los_results = unadjusted_icu_los_model.fit()
# print(unadjusted_icu_los_results.summary())

#### Ventilator days

In [ ]:
# ventilator days
ventilator_days_df = df[["Ventilator_[d]", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
ventilator_days_df.columns = ['ventilator_days', 'GCS', 'sex', 'age', 'category']
print(f'number of nan (excluded): {ventilator_days_df.isna().sum().sum()}')
ventilator_days_df.dropna(inplace=True)

overall_ventilator_days_model = Lmer(f"ventilator_days ~ GCS + sex + age + (1|category)", data=ventilator_days_df, family='gaussian')
overall_ventilator_days_res = overall_ventilator_days_model.fit()
overall_ventilator_days_res

In [ ]:
# survival model
# Here, we treat icu_survival == 1 (extubation) as the event, and icu_survival == 0 (death) as censored.
ventilator_days_df = df[["Ventilator_[d]", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded', 'ICU_Survival_[y/n]_encoded']]
ventilator_days_df.columns = ['ventilator_days', 'GCS', 'sex', 'age', 'category', 'icu_survival']
ventilator_days_df['icu_death'] = -1 * ventilator_days_df['icu_survival'] + 1
ventilator_days_df.drop(columns='icu_survival', inplace=True)
ventilator_days_df.dropna(inplace=True)

# Prepare the DataFrame for the model.
# We use 'ventilator_days' as the duration, 'event' as the event indicator, and include relevant covariates.
df_model = ventilator_days_df[['ventilator_days', 'icu_death', 'GCS', 'sex', 'age', 'category']].copy()

# Fit the cause-specific Cox model.
# In this model, death is treated as a censoring event.
cph = CoxPHFitter()
cph.fit(df_model, duration_col='ventilator_days', event_col='icu_death')
cph.print_summary()

In [ ]:
# unadjusted survival model
# Here, we treat icu_survival == 1 (extubation) as the event, and icu_survival == 0 (death) as censored.
ventilator_days_df = df[["Ventilator_[d]", 'GCS_(Erstbefund)', 'ICU_Survival_[y/n]_encoded']]
ventilator_days_df.columns = ['ventilator_days', 'GCS', 'icu_survival']
ventilator_days_df['event'] = ventilator_days_df['icu_survival'].apply(lambda x: 1 if x == 1 else 0)
ventilator_days_df.dropna(inplace=True)

# Prepare the DataFrame for the model.
# We use 'ventilator_days' as the duration, 'event' as the event indicator, and include relevant covariates.
df_model = ventilator_days_df[['ventilator_days', 'event', 'GCS']].copy()

# Fit the cause-specific Cox model.
# In this model, death is treated as a censoring event.
cph = CoxPHFitter()
cph.fit(df_model, duration_col='ventilator_days', event_col='event')
cph.print_summary()

In [ ]:
# unadjusted_ventilator_days_model = sm.OLS(ventilator_days_df['ventilator_days'], sm.add_constant(ventilator_days_df[['GCS']]))
# unadjusted_ventilator_days_results = unadjusted_ventilator_days_model.fit()
# print(unadjusted_ventilator_days_results.summary())

### Hospital LOS

In [ ]:
# hospital stay
# hospital_stay_df = df[["Hospital_stay_[d]", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
# hospital_stay_df.columns = ['hospital_stay', 'GCS', 'sex', 'age', 'category']
# print(f'number of nan (excluded): {hospital_stay_df.isna().sum().sum()}')
# hospital_stay_df.dropna(inplace=True)
#
# overall_hospital_stay_model = Lmer(f"hospital_stay ~ GCS + sex + age + (1|category)", data=hospital_stay_df, family='gaussian')
# overall_hospital_stay_res = overall_hospital_stay_model.fit()
# print(overall_hospital_stay_res)

In [ ]:
# survival model
hospital_stay_df = df[["Hospital_stay_[d]", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded', 'Spital_Survival_[y/n]_encoded']]
hospital_stay_df.columns = ['hospital_stay', 'GCS', 'sex', 'age', 'category', 'hospital_survival']
hospital_stay_df['hospital_death'] = -1 * hospital_stay_df['hospital_survival'] + 1
hospital_stay_df.drop(columns='hospital_survival', inplace=True)

print(f'number of nan (excluded): {hospital_stay_df.isna().sum().sum()}')
hospital_stay_df.dropna(inplace=True)
 
cph = CoxPHFitter()
cph.fit(hospital_stay_df, duration_col='hospital_stay', event_col='hospital_death')
cph.print_summary()

In [ ]:
# unadjusted survival model
hospital_stay_df = df[["Hospital_stay_[d]", 'GCS_(Erstbefund)', 'Spital_Survival_[y/n]_encoded']]
hospital_stay_df.columns = ['hospital_stay', 'GCS', 'hospital_survival']
hospital_stay_df['hospital_death'] = -1 * hospital_stay_df['hospital_survival'] + 1
hospital_stay_df.drop(columns='hospital_survival', inplace=True)
print(f'number of nan (excluded): {hospital_stay_df.isna().sum().sum()}')
hospital_stay_df.dropna(inplace=True)
 
cph = CoxPHFitter()
cph.fit(hospital_stay_df, duration_col='hospital_stay', event_col='hospital_death')
cph.print_summary()

In [ ]:
# unadjusted_hospital_stay_model = sm.OLS(hospital_stay_df['hospital_stay'], sm.add_constant(hospital_stay_df[['GCS']]))
# unadjusted_hospital_stay_results = unadjusted_hospital_stay_model.fit()
# print(unadjusted_hospital_stay_results.summary())

### 28 d Survival

In [ ]:
# 28 d Survival
# survival_df = df[["28_d_Survival_[y/n]_encoded", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
# survival_df.columns = ['survival', 'GCS', 'sex', 'age', 'category']
# print(f'number of nan (excluded): {survival_df.isna().sum().sum()}')
# survival_df.dropna(inplace=True)
#
# overall_survival_model = Lmer(f"survival ~ GCS + sex + age + (1|category)", data=survival_df, family='binomial')
# overall_survival_res = overall_survival_model.fit()
# print(overall_survival_res)
                    

In [ ]:
# multiple logistic regression
survival_df = df[["28_d_Survival_[y/n]_encoded", 'GCS_(Erstbefund)', 'SPO2_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
survival_df.columns = ['survival', 'GCS', 'spo2', 'sex', 'age', 'category']
print(f'number of nan (excluded): {survival_df.isna().sum().sum()}')
survival_df.dropna(inplace=True)

# If 'category' is a categorical variable, ensure it is treated as such.
survival_df['category'] = survival_df['category'].astype('category')

# Define the logistic regression formula.
# Adjusting for ventilator_days, along with GCS, sex, age, and category.
formula = "survival ~ GCS + sex + age + C(category)"

# Fit the logistic regression model.
model = smf.logit(formula, data=survival_df).fit()

# Print the summary of the model.
print(model.summary())


In [ ]:
unadjusted_survival_model = sm.Logit(survival_df['survival'], sm.add_constant(survival_df[['GCS']]))
unadjusted_survival_results = unadjusted_survival_model.fit()
print(unadjusted_survival_results.summary())

### Pneumonia

In [ ]:
# pneumonia
# pneumonia_df = df[['ITN_ass._Pneumonia_[y/n]_encoded', 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
# pneumonia_df.columns = ['pneumonia', 'GCS', 'sex', 'age', 'category']
# print(f'number of nan (excluded): {pneumonia_df.isna().sum().sum()}')
# pneumonia_df.dropna(inplace=True)
#
# overall_pneumonia_model = Lmer(f"pneumonia ~ GCS + sex + age + (1|category)", data=pneumonia_df, family='binomial')
# overall_pneumonia_res = overall_pneumonia_model.fit()
# print(overall_pneumonia_res)

In [ ]:
# multiple logistic regression

pneumonia_df = df[['ITN_ass._Pneumonia_[y/n]_encoded', 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded', 'Ventilator_[d]', 'Hospital_stay_[d]']]
pneumonia_df.columns = ['pneumonia', 'GCS', 'sex', 'age', 'category', 'ventilator_days', 'hospital_los']

# If 'category' is a categorical variable, ensure it is treated as such.
pneumonia_df['category'] = pneumonia_df['category'].astype('category')

# Define the logistic regression formula.
# Adjusting for ventilator_days, along with GCS, sex, age, and category.
formula = "pneumonia ~ GCS + sex + age + C(category) + hospital_los"

# Fit the logistic regression model.
model = smf.logit(formula, data=pneumonia_df).fit()

# Print the summary of the model.
print(model.summary())


In [ ]:
# unadjusted
pneumonia_df = df[['ITN_ass._Pneumonia_[y/n]_encoded', 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
pneumonia_df.columns = ['pneumonia', 'GCS', 'sex', 'age', 'category']
print(f'number of nan (excluded): {pneumonia_df.isna().sum().sum()}')
pneumonia_df.dropna(inplace=True)
unadjusted_pneumonia_model = sm.Logit(pneumonia_df['pneumonia'], sm.add_constant(pneumonia_df[['GCS']]))
unadjusted_pneumonia_results = unadjusted_pneumonia_model.fit()
print(unadjusted_pneumonia_results.summary())

In [ ]:
# save all model results in pdf with headings for each outcome and model

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io

# Function to add model results to PDF
def add_model_results_to_pdf(c, model_results,heading, start_y=750):
    c.setFont("Helvetica", 12)
    y_position = start_y
    c.drawString(100, y_position, heading)
    y_position -= 20 
    
    text_object = c.beginText(40, y_position)
    text_object.setFont("Helvetica", 10)
    for line in model_results.split('\n'):
        text_object.textLine(line)
        y_position -= 10
        # Check if we need to start a new page
        if y_position < 40:  # Adjust based on your footer size
            c.drawText(text_object)
            c.showPage()  # Start a new page
            c.setFont("Helvetica", 12)
            y_position = start_y - 20  # Reset y_position for the new page
            text_object = c.beginText(40, y_position)
            text_object.setFont("Helvetica", 10)

    c.drawText(text_object)

# Create a new PDF with ReportLab
c = canvas.Canvas("/Users/jk1/Downloads/model_results.pdf", pagesize=letter)
height = letter[1]

# Assuming you have model results as strings, for example:
overall_icu_los_res_str = str(overall_icu_los_res)
unadjusted_icu_los_results_str = str(unadjusted_icu_los_results.summary())
# Add more model results strings as needed

add_model_results_to_pdf(c, overall_icu_los_res_str, "Adjusted ICU LOS Model Results")
c.showPage()  # Start a new page for the next model if necessary
add_model_results_to_pdf(c, unadjusted_icu_los_results_str, "Unadjusted ICU LOS Model Results")
c.showPage()  # Start a new page for the next model if necessary


add_model_results_to_pdf(c, str(overall_ventilator_days_res), "Adjusted Ventilator Days Model Results")
c.showPage()  # Start a new page for the next model if necessary
add_model_results_to_pdf(c, str(unadjusted_ventilator_days_results.summary()), "Unadjusted Ventilator Days Model Results")
c.showPage()  # Start a new page for the next model if necessary

add_model_results_to_pdf(c, str(overall_hospital_stay_res), "Adjusted Hospital Stay Model Results")
c.showPage()  # Start a new page for the next model if necessary
add_model_results_to_pdf(c, str(unadjusted_hospital_stay_results.summary()), "Unadjusted Hospital Stay Model Results")
c.showPage()  # Start a new page for the next model if necessary

add_model_results_to_pdf(c, str(overall_survival_res), "Adjusted 28 d Survival Model Results")
c.showPage()  # Start a new page for the next model if necessary
add_model_results_to_pdf(c, str(unadjusted_survival_results.summary()), "Unadjusted 28 d Survival Model Results")
c.showPage()  # Start a new page for the next model if necessary

add_model_results_to_pdf(c, str(overall_pneumonia_res), "Adjusted Pneumonia Model Results")
c.showPage()  # Start a new page for the next model if necessary
add_model_results_to_pdf(c, str(unadjusted_pneumonia_results.summary()), "Unadjusted Pneumonia Model Results")

# Save the PDF
c.save()


## Impact of Category on long-term outcomes

In [ ]:
df

In [ ]:
# Pneumonia
res_pneumonia = rank_compare_2indep(
    df[(df['Einteilung_(Hauptdiagnose)'] == 'Krankheit') & (df['ITN_ass._Pneumonia_[y/n]_encoded'].notna())]['ITN_ass._Pneumonia_[y/n]_encoded'].values,
    df[(df['Einteilung_(Hauptdiagnose)'] == 'Unfall') & (df['ITN_ass._Pneumonia_[y/n]_encoded'].notna())]['ITN_ass._Pneumonia_[y/n]_encoded'].values)

In [ ]:
res_pneumonia.pvalue

In [ ]:
# ventilator days Ventilator_[d]
res_ventilator_days = rank_compare_2indep(
    df[(df['Einteilung_(Hauptdiagnose)'] == 'Krankheit') & (df['Ventilator_[d]'].notna())]['Ventilator_[d]'].values,
    df[(df['Einteilung_(Hauptdiagnose)'] == 'Unfall') & (df['Ventilator_[d]'].notna())]['Ventilator_[d]'].values)


In [ ]:
res_ventilator_days.pvalue

In [ ]:
# icu stay - ICU_[d]
res_icu_los = rank_compare_2indep(
    df[(df['Einteilung_(Hauptdiagnose)'] == 'Krankheit') & (df['ICU_[d]'].notna())]['ICU_[d]'].values,
    df[(df['Einteilung_(Hauptdiagnose)'] == 'Unfall') & (df['ICU_[d]'].notna())]['ICU_[d]'].values)



In [ ]:
res_icu_los.pvalue

In [ ]:
# hospital los - Hospital_stay_[d]
res_hospital_los = rank_compare_2indep(
    df[(df['Einteilung_(Hauptdiagnose)'] == 'Krankheit') & (df['Hospital_stay_[d]'].notna())]['Hospital_stay_[d]'].values,
    df[(df['Einteilung_(Hauptdiagnose)'] == 'Unfall') & (df['Hospital_stay_[d]'].notna())]['Hospital_stay_[d]'].values)


In [ ]:
res_hospital_los.pvalue

In [ ]:
# survival - 28_d_Survival_[y/n]_encoded
res_survival = rank_compare_2indep(
    df[(df['Einteilung_(Hauptdiagnose)'] == 'Krankheit') & (df['28_d_Survival_[y/n]_encoded'].notna())]['28_d_Survival_[y/n]_encoded'].values,
    df[(df['Einteilung_(Hauptdiagnose)'] == 'Unfall') & (df['28_d_Survival_[y/n]_encoded'].notna())]['28_d_Survival_[y/n]_encoded'].values)

In [ ]:
res_survival.pvalue

In [ ]:
# table 
impact_of_category_on_outcomes = pd.DataFrame({
    'Outcome': ['Pneumonia', 'Ventilator days', 'ICU stay', 'Hospital stay', 'Survival'],
    'p-value': [res_pneumonia.pvalue, res_ventilator_days.pvalue, res_icu_los.pvalue, res_hospital_los.pvalue, res_survival.pvalue]
})

In [ ]:
import statsmodels.stats.multitest

adjustements = statsmodels.stats.multitest.multipletests(impact_of_category_on_outcomes['p-value'], method='fdr_bh')
adjusted_pvals = adjustements[1] 
impact_of_category_on_outcomes['adjusted_p-value'] = adjusted_pvals

In [ ]:
impact_of_category_on_outcomes

## Impact of sex on long-term outcomes


In [ ]:
# Pneumonia
sex_res_pneumonia = rank_compare_2indep(
       df[(df['Sverweis_Geschlecht_encoded'] == 1) & (df['ITN_ass._Pneumonia_[y/n]_encoded'].notna())]['ITN_ass._Pneumonia_[y/n]_encoded'].values,
       df[(df['Sverweis_Geschlecht_encoded'] == 0) & (df['ITN_ass._Pneumonia_[y/n]_encoded'].notna())]['ITN_ass._Pneumonia_[y/n]_encoded'].values)
sex_res_pneumonia.pvalue

In [ ]:
# ventilator days Ventilator_[d]
sex_res_ventilator_days = rank_compare_2indep(
    df[(df['Sverweis_Geschlecht_encoded'] == 1) & (df['Ventilator_[d]'].notna())]['Ventilator_[d]'].values,
    df[(df['Sverweis_Geschlecht_encoded'] == 0) & (df['Ventilator_[d]'].notna())]['Ventilator_[d]'].values)

sex_res_ventilator_days.pvalue

In [ ]:
# icu stay - ICU_[d]
sex_res_icu_los = rank_compare_2indep(
    df[(df['Sverweis_Geschlecht_encoded'] == 1) & (df['ICU_[d]'].notna())]['ICU_[d]'].values,
    df[(df['Sverweis_Geschlecht_encoded'] == 0) & (df['ICU_[d]'].notna())]['ICU_[d]'].values)

sex_res_icu_los.pvalue

In [ ]:
# hospital los - Hospital_stay_[d]
sex_res_hospital_los = rank_compare_2indep(
    df[(df['Sverweis_Geschlecht_encoded'] == 1) & (df['Hospital_stay_[d]'].notna())][
        'Hospital_stay_[d]'].values,
    df[(df['Sverweis_Geschlecht_encoded'] == 0) & (df['Hospital_stay_[d]'].notna())]['Hospital_stay_[d]'].values)

sex_res_hospital_los.pvalue

In [ ]:
# survival - 28_d_Survival_[y/n]_encoded
sex_res_survival = rank_compare_2indep(
    df[(df['Sverweis_Geschlecht_encoded'] == 1) & (df['28_d_Survival_[y/n]_encoded'].notna())][
        '28_d_Survival_[y/n]_encoded'].values,
    df[(df['Sverweis_Geschlecht_encoded'] == 0) & (df['28_d_Survival_[y/n]_encoded'].notna())][
        '28_d_Survival_[y/n]_encoded'].values)
sex_res_survival.pvalue

In [ ]:
# table 
impact_of_sex_on_outcomes = pd.DataFrame({
    'Outcome': ['Pneumonia', 'Ventilator days', 'ICU stay', 'Hospital stay', 'Survival'],
    'p-value': [sex_res_pneumonia.pvalue, sex_res_ventilator_days.pvalue, sex_res_icu_los.pvalue, sex_res_hospital_los.pvalue, sex_res_survival.pvalue]
})

In [ ]:
import statsmodels.stats.multitest

sex_adjustements = statsmodels.stats.multitest.multipletests(impact_of_sex_on_outcomes['p-value'], method='fdr_bh')
sex_adjusted_pvals = sex_adjustements[1]
impact_of_sex_on_outcomes['multi_test_adjusted_p-value'] = sex_adjusted_pvals

In [ ]:
impact_of_sex_on_outcomes

## Impact of SpO2 on long-term outcomes

In [ ]:
icu_los_df = df[["ICU_[d]", 'SPO2_(Erstbefund)']]
print(f'number of nan (excluded): {icu_los_df.isna().sum().sum()}')
icu_los_df.dropna(inplace=True)


unadjusted_icu_los_model = sm.OLS(icu_los_df['ICU_[d]'], sm.add_constant(icu_los_df[['SPO2_(Erstbefund)']]))
unadjusted_icu_los_results = unadjusted_icu_los_model.fit()
print(unadjusted_icu_los_results.summary())

In [ ]:
multi_var_icu_los_df = df[["ICU_[d]", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded', 'SPO2_(Erstbefund)']]
multi_var_icu_los_df.columns = ['ICU_los', 'GCS', 'sex', 'age', 'category', 'SPO2']
print(f'number of nan (excluded): {multi_var_icu_los_df.isna().sum().sum()}')
multi_var_icu_los_df.dropna(inplace=True)


multi_var_icu_los_model = Lmer(f"ICU_los ~ SPO2 + GCS + sex + age + (1|category)", data=multi_var_icu_los_df, family='gaussian')
multi_var_icu_los_res = multi_var_icu_los_model.fit()
print(multi_var_icu_los_res)

In [ ]:
# 28 d Survival
survival_df = df[["28_d_Survival_[y/n]_encoded", 'SPO2_(Erstbefund)']]
survival_df.columns = ['survival', 'SPO2']
print(f'number of nan (excluded): {survival_df.isna().sum().sum()}')
survival_df.dropna(inplace=True)

unadjusted_survival_model = sm.Logit(survival_df['survival'], sm.add_constant(survival_df[['SPO2']]))
unadjusted_survival_results = unadjusted_survival_model.fit()
print(unadjusted_survival_results.summary())

In [ ]:
survival_df = df[["28_d_Survival_[y/n]_encoded", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded', 'SPO2_(Erstbefund)']]
survival_df.columns = ['survival', 'GCS', 'sex', 'age', 'category', 'SPO2']
print(f'number of nan (excluded): {survival_df.isna().sum().sum()}')
survival_df.dropna(inplace=True)

multi_var_survival_model = Lmer(f"survival ~ SPO2 + GCS + sex + age + (1|category)", data=survival_df, family='binomial')
multi_var_survival_model = multi_var_survival_model.fit()
print(multi_var_survival_model)